# Before Dataclasses
- tuple, dict, attrs, namedtuple
- Remember: tuples are immutable
- Dict does not allow access with .notation
- attrs 3rd party library
- See: https://realpython.com/python-data-classes/

In [5]:
queen_of_hearts_tuple = ('Q', 'Hearts')
queen_of_hearts_dict = {'rank': 'Q', 'suit': 'Hearts'}

print(queen_of_hearts_tuple[0])  # No named access, immutable
print(queen_of_hearts_dict['suit'])  # Would be nicer with .suit

Q
Hearts


In [6]:
from collections import namedtuple

NamedTupleCard = namedtuple('NamedTupleCard', ['rank', 'suit'])

queen_of_hearts = NamedTupleCard('Q', 'Hearts')
queen_of_hearts.rank

'Q'

In [10]:
import attr

@attr.s
class AttrsCard:
    rank = attr.ib()
    suit = attr.ib()

card1 = AttrsCard(rank='Q', suit='Hearts')
card1

AttrsCard(rank='Q', suit='Hearts')

# Value of Dataclasses
- Python 3.7
- Default Values (default, default_factory)
- Type annotations (Any, List)
- field, fields
- Inheritance (be careful about default values)
- immutable with frozen (like namedtuple)
- comparsion with order 
- Default implementation for repr, init, compare
- make_dataclass to dynamically define dataclass
- metadata
- .Notation instead of dict access like ['text']
- work with json (pip install dataclasses-json)

In [11]:
# Basics

from dataclasses import dataclass

@dataclass
class Position:
    name: str
    lon: float
    lat: float

pos = Position('Oslo', 10.8, 59.9)
print(pos)
Position(name='Oslo', lon=10.8, lat=59.9)
pos.lat

Position(name='Oslo', lon=10.8, lat=59.9)


59.9

In [12]:
# Dynamic creation of Dataclass

from dataclasses import make_dataclass

Position = make_dataclass('Position', ['name', 'lat', 'lon'])


In [13]:
# Default Values

from dataclasses import dataclass

@dataclass
class Position:
    name: str
    lon: float = 0.0
    lat: float = 0.0

Position('Null Island')
Position(name='Null Island', lon=0.0, lat=0.0)
Position('Greenwich', lat=51.8)
Position(name='Greenwich', lon=0.0, lat=51.8)
Position('Vancouver', -123.1, 49.3)

Position(name='Vancouver', lon=-123.1, lat=49.3)

In [14]:
# Any Type

from dataclasses import dataclass
from typing import Any

@dataclass
class WithoutExplicitTypes:
    name: Any
    value: Any = 42

In [17]:
# Typing List + Dataclasses

from dataclasses import dataclass
from typing import List

@dataclass
class PlayingCard:
    rank: str
    suit: str

@dataclass
class Deck:
    cards: List[PlayingCard]

queen_of_hearts = PlayingCard('Q', 'Hearts')
ace_of_spades = PlayingCard('A', 'Spades')
two_cards = Deck([queen_of_hearts, ace_of_spades])
print(two_cards)

Deck(cards=[PlayingCard(rank='Q', suit='Hearts'), PlayingCard(rank='A', suit='Spades')])


In [19]:
# List comprehension with multiple For-Loops
# Factory method

RANKS = '2 3 4 5 6 7 8 9 10 J Q K A'.split()
SUITS = '♣ ♢ ♡ ♠'.split()

def make_french_deck():
    return [PlayingCard(r, s) for s in SUITS for r in RANKS]

from dataclasses import dataclass, field
from typing import List

@dataclass
class Deck:  
    # cards: List[PlayingCard] = make_french_deck() --> # Will NOT work, every instance of deck will refer to the same list!!!
    cards: List[PlayingCard] = field(default_factory=make_french_deck)

Deck()


Deck(cards=[PlayingCard(rank='2', suit='♣'), PlayingCard(rank='3', suit='♣'), PlayingCard(rank='4', suit='♣'), PlayingCard(rank='5', suit='♣'), PlayingCard(rank='6', suit='♣'), PlayingCard(rank='7', suit='♣'), PlayingCard(rank='8', suit='♣'), PlayingCard(rank='9', suit='♣'), PlayingCard(rank='10', suit='♣'), PlayingCard(rank='J', suit='♣'), PlayingCard(rank='Q', suit='♣'), PlayingCard(rank='K', suit='♣'), PlayingCard(rank='A', suit='♣'), PlayingCard(rank='2', suit='♢'), PlayingCard(rank='3', suit='♢'), PlayingCard(rank='4', suit='♢'), PlayingCard(rank='5', suit='♢'), PlayingCard(rank='6', suit='♢'), PlayingCard(rank='7', suit='♢'), PlayingCard(rank='8', suit='♢'), PlayingCard(rank='9', suit='♢'), PlayingCard(rank='10', suit='♢'), PlayingCard(rank='J', suit='♢'), PlayingCard(rank='Q', suit='♢'), PlayingCard(rank='K', suit='♢'), PlayingCard(rank='A', suit='♢'), PlayingCard(rank='2', suit='♡'), PlayingCard(rank='3', suit='♡'), PlayingCard(rank='4', suit='♡'), PlayingCard(rank='5', suit='♡

In [22]:
# Metadata

from dataclasses import dataclass, field, fields

@dataclass
class Position:
    name: str
    lon: float = field(default=0.0, metadata={'unit': 'degrees'})
    lat: float = field(default=0.0, metadata={'unit': 'degrees'})

fields(Position)
lat_unit = fields(Position)[2].metadata['unit']
lat_unit

'degrees'

In [30]:
# Using !s in F-String for string representation of another object

from typing import List

@dataclass
class PlayingCard:
    rank: str
    suit: str

    def __str__(self):
        return f'{self.suit}{self.rank}'

@dataclass
class Deck:
    cards: List[PlayingCard] = field(default_factory=make_french_deck)

    def __repr__(self):
        cards = ', '.join(f'{c!s}' for c in self.cards)
        return f'{self.__class__.__name__}({cards})'

Deck().__repr__()

'Deck(♣2, ♣3, ♣4, ♣5, ♣6, ♣7, ♣8, ♣9, ♣10, ♣J, ♣Q, ♣K, ♣A, ♢2, ♢3, ♢4, ♢5, ♢6, ♢7, ♢8, ♢9, ♢10, ♢J, ♢Q, ♢K, ♢A, ♡2, ♡3, ♡4, ♡5, ♡6, ♡7, ♡8, ♡9, ♡10, ♡J, ♡Q, ♡K, ♡A, ♠2, ♠3, ♠4, ♠5, ♠6, ♠7, ♠8, ♠9, ♠10, ♠J, ♠Q, ♠K, ♠A)'

In [31]:
# Order to implement comparsion <, > etc.
# .sort_index is added as the first field of the class. That way, the comparison is first done using .sort_index and 
#   only if there are ties are the other fields used
# __post_init__() to calculated value from the other fields .rank and .suit automatically
from dataclasses import dataclass

@dataclass(order=True)
class PlayingCard:
    sort_index: int = field(init=False, repr=False)
    rank: str
    suit: str

    def __post_init__(self):
        self.sort_index = (RANKS.index(self.rank) * len(SUITS)
                           + SUITS.index(self.suit))

    def __str__(self):
        return f'{self.suit}{self.rank}'

queen_of_hearts = PlayingCard('Q', '♡')
ace_of_spades = PlayingCard('A', '♠')
ace_of_spades > queen_of_hearts

True

In [32]:
# values can then be sorted
Deck(sorted(make_french_deck()))

Deck(♣2, ♢2, ♡2, ♠2, ♣3, ♢3, ♡3, ♠3, ♣4, ♢4, ♡4, ♠4, ♣5, ♢5, ♡5, ♠5, ♣6, ♢6, ♡6, ♠6, ♣7, ♢7, ♡7, ♠7, ♣8, ♢8, ♡8, ♠8, ♣9, ♢9, ♡9, ♠9, ♣10, ♢10, ♡10, ♠10, ♣J, ♢J, ♡J, ♠J, ♣Q, ♢Q, ♡Q, ♠Q, ♣K, ♢K, ♡K, ♠K, ♣A, ♢A, ♡A, ♠A)

In [33]:
from random import sample
Deck(sample(make_french_deck(), k=10))

Deck(♢4, ♡4, ♠9, ♡7, ♠2, ♠6, ♣7, ♢2, ♡K, ♣9)

In [ ]:
# immutable

from dataclasses import dataclass

@dataclass(frozen=True)
class Position:
    name: str
    lon: float = 0.0
    lat: float = 0.0

In [34]:
# Inheritance
#  Be careful about default attributes

from dataclasses import dataclass

@dataclass
class Position:
    name: str
    lon: float = 0.0
    lat: float = 0.0

@dataclass
class Capital(Position):
    #country: str  # Does NOT work
    # as this is not allowed in python
    # def __init__(name: str, lon: float = 0.0, lat: float = 0.0, country: str):
    country: str = 'Unknown'
    lat: float = 40.0

Capital('Madrid', country='Spain')

Capital(name='Madrid', lon=0.0, lat=40.0, country='Spain')